# Initial EDA 

Initial EDA for the dataset to have a look at the data structure, trends, costumers and products, order baskets etc. and to aquire ideas for segmentation

## Imports

In [1]:
import os

# ETL and Data Manipulation
import pandas as pd
import numpy as np

# Visualizations
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

## Data Overview

### Load data from CSV

In [2]:
DATA_PATH = os.path.join('..', '..', 'data')

order_lines = pd.read_csv(os.path.join(DATA_PATH, 'orders.csv'))
products = pd.read_csv(os.path.join(DATA_PATH, 'product-supplier.csv'))

### Check data types

#### Datetime columns

In [3]:
date_str_format = '%d-%b-%y'

order_lines['Date Order was placed'] = pd.to_datetime(order_lines['Date Order was placed'], format = date_str_format)
order_lines['Delivery Date'] = pd.to_datetime(order_lines['Delivery Date'], format = date_str_format)

#### Customer Status column 

Has mixed lower and upper case

In [4]:
order_lines['Customer Status'] = order_lines['Customer Status'].str.lower().astype(str)

#### Retail price per unit

We have cost price per unit and could probably use the retail price per unit as well

In [5]:
order_lines['Retail price Per Unit'] = order_lines['Total Retail Price for This Order'] / order_lines['Quantity Ordered']

#### Total cost price

We have cost price per unit, so we also calculate the total cost price

In [6]:
order_lines['Total Cost price'] = order_lines['Quantity Ordered'] * order_lines['Cost Price Per Unit']

#### Delivery time

How long has the delivery time been

In [7]:
order_lines['Delivery time'] = (order_lines['Delivery Date'] - order_lines['Date Order was placed']).dt.days

## Revenue Trends Over Time

#### Total revenue over time

In [8]:
# Aggregate over months
order_lines_monthly = order_lines.join(products.set_index('Product ID'), on='Product ID', how='left').set_index('Date Order was placed').groupby([pd.Grouper(freq='ME'), 'Product Line'])
monthly_revenue = order_lines_monthly.agg({'Total Cost price' : 'sum'}).reset_index()

# Plot
fig = px.area(monthly_revenue, x='Date Order was placed', y='Total Cost price', color='Product Line')
fig.show()

##### 💡 All categories have a strong seasonal trend with peaks twice a year (summer and xmas)

## Customers

### Turnover per customer

In [9]:
# Group by customer
order_lines_per_costumer = order_lines.groupby('Customer ID').agg({'Total Cost price' : 'sum'})

# Show distribution
px.histogram(order_lines_per_costumer, range_x=(0,order_lines_per_costumer['Total Cost price'].quantile(0.95)), title='Turnover per customer')

### Customers frequency of buying

In [10]:
order_lines_per_costumer = order_lines.groupby('Customer ID')['Order ID'].nunique()
px.histogram(order_lines_per_costumer)

##### 💡 There are many small customers and the vast majority has placed < 10 orders in total

## Products

### Profit Margins per category

In [11]:
# Add product information
order_lines_products = order_lines.join(products.set_index('Product ID'), on='Product ID', how='left').set_index('Date Order was placed')

# Group by product category
order_lines_products_per_category = order_lines_products.groupby('Product Category')
order_lines_products_per_category_mean = order_lines_products_per_category.agg({'Cost Price Per Unit' : 'mean', 'Retail price Per Unit' : 'mean'})

# Add column with margin
order_lines_products_per_category_mean['Profit Margin'] = order_lines_products_per_category_mean['Retail price Per Unit'] - order_lines_products_per_category_mean['Cost Price Per Unit']
order_lines_products_per_category_mean['Profit Margin Rate'] = order_lines_products_per_category_mean['Profit Margin'] / order_lines_products_per_category_mean['Retail price Per Unit']

# Plot 
order_lines_products_per_category_mean = order_lines_products_per_category_mean.sort_values('Profit Margin Rate', ascending=False)
px.bar(order_lines_products_per_category_mean, x=order_lines_products_per_category_mean.index, y='Profit Margin Rate', title='Profit Margin per Category')

##### 💡 The margin (percentage-wise) across categories are similar 

### Product Category sizes

In [12]:
# Aggregate the grouped order lines to get size of the category
order_lines_products_per_category = order_lines_products.groupby(['Product Category','Product Line'])

order_lines_products_per_category_sum = order_lines_products_per_category.agg({'Total Cost price' : 'sum'}).reset_index()

# Plot
order_lines_products_per_category_sum = order_lines_products_per_category_sum.sort_values('Total Cost price', ascending=False)
px.bar(order_lines_products_per_category_sum, x='Product Category', y='Total Cost price', color='Product Line', color_discrete_sequence=px.colors.qualitative.Plotly)

##### 💡 Outdoors and children sports are a single-category product line, clothes & shoes have two categories with similar size and sports are split between 8 categories


## Order Basket Analysis

### Total order value distribution

In [ ]:
# Group by order ID
order_lines_orders = order_lines.set_index('Date Order was placed').groupby([pd.Grouper(freq='YE'),'Order ID'])

# Get total cost price per order ID
order_lines_orders_value = order_lines_orders.agg({'Total Cost price' : 'sum'}).reset_index()

# Plot
px.histogram(order_lines_orders_value, 
             x='Total Cost price', 
             color='Date Order was placed', 
             range_x=(0,order_lines_orders_value['Total Cost price'].quantile(0.95)),
             title='Total basket volume')

##### 💡 The total basket volume are comparable across years